# Uebung 4 Mehrklassenklassifikation mit Neuronalen Netzen


## Imports

In [2]:
import os 
import random
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from skimage.transform import rescale, resize
from sklearn.preprocessing import OneHotEncoder

import csv
import cv2
import numpy as np
import pandas as pd
import scipy as scp 
import matplotlib.pyplot as plt
from IPython.display import display
from ipywidgets import interact, interactive, fixed, interact_manual, widgets


## Globale Variablen
Definiere an dieser Stelle alle Variablen, die global verwendet werden, z.B.: Pfadnamen

In [18]:
def readTrafficSigns(rootpath, classes=None, resize_shape=None):
    """
    Reads traffic sign data for German Traffic Sign Recognition Benchmark.

    Arguments: path to the traffic sign data, for example './GTSRB/Training'
            classes intended to be read
    Returns:   list of images, list of corresponding labels
    
    """
    
    images = []
    labels = []
    
    # for c in range(0,43):

    if classes is None:
        # loop over all 42 classes
        classes = [i for i in range(0,43)]
    
    for c in classes:
        # subdirectory for class
        prefix = rootpath + '/' + format(c, '05d') + '/'
        
        # annotations file
        gtFile = open(prefix + 'GT-'+ format(c, '05d') + '.csv')
        
        # csv parser for annotations file
        gtReader = csv.reader(gtFile, delimiter=';')
        
        # Skip header
        next(gtReader)
        
        # loop over all images in current annotations file
        for row in gtReader:
            # the 1th column is the filename
            img = plt.imread(prefix + row[0])
            if resize_shape is not None:
                    img = tf.image.convert_image_dtype(img, tf.float32) # equivalent to dividing image pixels by 255
                    img = tf.image.resize(img, resize_shape) # Resizing the image to 224x224 dimention
            
            images.append(img)
            
            # the 8th column is the label
            labels.append(int(row[7]))
            
        gtFile.close()
    
    # Convert list of labels to array of labels
    labels = np.array(labels)
    
    return images, labels

def getImagesByClassID(all_images, all_labels, class_id):
    """
    Gets images by class id from the list of all images
    
    """
    # Get index for the elements that match with the specified class id
    class_id_idx = np.where(all_labels == class_id)[0]
    
    # Get images and labels for the two classes
    labels = list(all_labels)[class_id_idx.min() : class_id_idx.max()]
    images = all_images[class_id_idx.min() : class_id_idx.max()]
    
    return images, labels

def randomlyMixDatasets(data_sorted, labels_sorted):
    """
    Randomly mixes sorted data and label lists in the same way
    
    """
    
    combined_set = list(zip(data_sorted, labels_sorted))
    random.shuffle(combined_set)
    data, labels = zip(*combined_set)
    
    return data, labels

def onehotencoderlabels(labels):
    b = np.zeros((labels.size, labels.max()+1))
    b[np.arange(labels.size),labels] = 1
    idx = np.argwhere(np.all(b[..., :] == 0, axis=0))
    b = np.delete(b, idx, axis=1)
    return b

In [6]:
DATA_PATH = f"{os.environ['HOME']}/Documents/TU Berlin/10 SS22/BGA II/UBs/data"
cl = [random.randint(0, 42) for _ in range(2)]
CLASSES = [str('%05d' % i) for i in cl]
images_folders = ['GTSRB_Final_Training_Images/GTSRB/Final_Training/Images',
    'GTSRB_Final_Test_Images/GTSRB/Final_Test/Images', # + Classes Folder
    ]
hog_paths = ['GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_01',
    'GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_02',
    'GTSRB_Final_Test_HOG/GTSRB/Final_Test/HOG/HOG_03',
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_01', # + Classes Folder
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_02', # + Classes Folder
    'GTSRB_Final_Training_HOG/GTSRB/Final_Training/HOG/HOG_03', # + Classes Folder
    ]
TRAIN_PATH = os.path.join(DATA_PATH, "GTSRB_Final_Training_Images/GTSRB/Final_Training/Images")
TEST_PATH = os.path.join(DATA_PATH, "GTSRB_Final_Test_Images/GTSRB/Final_Test/Images")
print(CLASSES)

['00001', '00007']


## Datenaufbereitung
Hinweise findest du hier: https://keras.io/getting_started/intro_to_keras_for_engineers/#data-loading-amp-preprocessing

In [7]:
# Read all the Traffic Sign data
all_images, all_labels = readTrafficSigns(TRAIN_PATH, cl, (30, 30))

print("len(all_images) = {}".format(len(all_images)))
print("all_labels.shape = {}".format(all_labels.shape))

2022-06-23 16:08:16.014286: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-23 16:08:16.091160: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-23 16:08:16.174000: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


len(all_images) = 3660
all_labels.shape = (3660,)


In [32]:
all_images = np.array(all_images)
all_images.shape

(3660, 30, 30, 3)

In [24]:
all_labels = onehotencoderlabels(all_labels)
print("all_labels.shape = {}".format(all_labels.shape))

all_labels.shape = (3660, 2)


In [25]:
print("type(all_images[0]) = {}".format(type(all_labels)))
print("all_images[0].shape = {}".format(all_images[0].shape))

count = 0
img_shape_0 = []
img_shape_1 = []
img_shape_2 = []
for i,a in enumerate(all_images):
  img_shape_0.append(a.shape[0])
  img_shape_1.append(a.shape[1])
  img_shape_2.append(a.shape[2])

print(np.unique(img_shape_0))
print(np.unique(img_shape_1))
print(np.unique(img_shape_2))


type(all_images[0]) = <class 'numpy.ndarray'>
all_images[0].shape = (30, 30, 3)
[30]
[30]
[3]


In [45]:
def show_img(idx):
    plt.figure(figsize=(4,4))
    plt.imshow(all_images[idx])
    plt.axis('off')
    plt.show()

interact(show_img, idx=widgets.IntSlider(min=0,max=len(all_images), step=1, value=0));

interactive(children=(IntSlider(value=0, description='idx', max=1170), Output()), _dom_classes=('widget-intera…

## Aufbau des Modells
Zum Aufbau deines Modells kannst du dich an die gezeigten Beispiele richten. Implementiere zuerst ein einfaches Modell, welches du je nach Performance erweitern kannst. 

Unten findest du die Auflistung der Schichten (Layers), die du für dein Modell miteinander kombinieren kannst. 


Überlege dir, welche Layers für die Klassifikationsaufgabe mit HOG-Features gut sind und welche Layer sich für die Klassifikationsaufgabe mit ppm-Dateien eignen.

In [27]:
input_shape = (30, 30, 3)
model = keras.Sequential()
model.add(keras.Input(shape=input_shape))
model.add(keras.layers.Conv2D(20, 20, activation='relu'))
model.add(keras.layers.Conv2D(5, 5, activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(2))
model.add(keras.layers.Activation('softmax'))

## Kompilieren des Modells
Eine detaillierte Beschreibung der [compile](https://keras.io/api/models/model_training_apis/#compile-method)-Methode findest du in Keras API Referenz.

In [35]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
            #   loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=[tf.keras.metrics.BinaryAccuracy(),
                       tf.keras.metrics.FalseNegatives()])

## Training des Modells
Eine detaillierte Beschreibung der [fit](https://keras.io/api/models/model_training_apis/#fit-method)-Methode findest du in Keras API Referenz.

In [33]:
X_train, X_test, y_train, y_test = train_test_split(all_images, all_labels, test_size=0.25, random_state=42)

In [39]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
86/86 [==============================] - 10s 116ms/step - loss: 0.3634 - binary_accuracy: 0.8725 - false_negatives_1: 350.0000
Epoch 2/5
86/86 [==============================] - 8s 95ms/step - loss: 0.2496 - binary_accuracy: 0.9290 - false_negatives_1: 195.0000
Epoch 3/5
86/86 [==============================] - 8s 91ms/step - loss: 0.1922 - binary_accuracy: 0.9468 - false_negatives_1: 146.0000
Epoch 4/5
86/86 [==============================] - 9s 106ms/step - loss: 0.1359 - binary_accuracy: 0.9650 - false_negatives_1: 96.0000
Epoch 5/5
86/86 [==============================] - 9s 109ms/step - loss: 0.1306 - binary_accuracy: 0.9599 - false_negatives_1: 110.0000


## Evaluation des Modells
Eine detaillierte Beschreibung der [evaluate](https://keras.io/api/models/model_training_apis/#evaluate-method)-Methode findest du in Keras API Referenz.

Nach der Anwendung der *evaluate*-Methode kannst du dir zusätzlich den ausfuehrlichen Klassifikationsbericht (*classification_report()*) sowie die Konfusionsmatrix (*confusion_matrix()*) anschauen.  

In [40]:
model.evaluate(X_test, y_test)

29/29 [==============================] - 1s 23ms/step - loss: 0.1094 - binary_accuracy: 0.9749 - false_negatives_1: 23.0000


[0.10940083861351013, 0.9748634099960327, 23.0]

## Speichern des trainierten Modells
Zum Speichern des trainierten Modells kann *save*-Methode
Weiterfuehrende Informationen zu dieser Methode unter folgendem [Link](https://keras.io/api/models/model_saving_apis/) zu finden. 

In [41]:
model.save("./model.h5")

## Nutzen des trainierten Modells zum Vorhersagen von Verkehrszeichen-Klassen 
Zum Wiederverwenden des trainierten gespeicherten Modells kann die [load_model](https://keras.io/api/models/model_saving_apis/#loadmodel-function)-Funktionverwendet werden.

Eine detaillierte Beschreibung der [predict](https://keras.io/api/models/model_training_apis/#predict-method)-Methode findest du in Keras API Referenz.

In [42]:
tmp_model = keras.models.load_model("./model.h5")
tmp_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 11, 11, 20)        24020     
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 5)           2505      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 5)           0         
_________________________________________________________________
dropout (Dropout)            (None, 3, 3, 5)           0         
_________________________________________________________________
flatten (Flatten)            (None, 45)                0         
_________________________________________________________________
dense (Dense)                (None, 2)                 92        
_________________________________________________________________
activation (Activation)      (None, 2)                 0